## **1. Google Drive**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#cur_dir = "/content/drive/MyDrive/CH2/Notebooks"
#%cd $cur_dir


In [ ]:
#%pip install torchview


## **2. Import Libraries**

In [ ]:
# Set seed for reproducibility
SEED =42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torchvision
#from torchvision.transforms import v2 as transforms
from torch.utils.data import TensorDataset, DataLoader
from torchview import draw_graph
from scipy import ndimage
from PIL import Image
import torch.nn.functional as F
from torchvision import models
from torchvision import transforms as tfs
import math
from transformers import AutoModel
# Configurazione di TensorBoard e directory
logs_dir = "tensorboard"
!pkill -f tensorboard
%load_ext tensorboard
!mkdir -p models

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import cv2
import copy
import shutil
from itertools import product
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.gridspec as gridspec
import requests
from io import BytesIO
from tqdm import tqdm
import glob
from pathlib import Path
import shutil
import gc
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import torch.nn.functional as F

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

## **3. Config**

In [ ]:
USE_MASKED_PATCHES = False

In [ ]:
datasets_path = os.path.join(os.path.pardir, "an2dl2526c2")

train_data_path = os.path.join(datasets_path, "train_data")
train_labels_path = os.path.join(datasets_path, "train_labels.csv")
test_data_path = os.path.join(datasets_path, "test_data")

CSV_PATH = train_labels_path                # Path to the CSV file with labels
SOURCE_FOLDER = train_data_path

if USE_MASKED_PATCHES:
  PATCHES_OUT = os.path.join(datasets_path, "preprocessing_results_masked","train_patches_masked")
  SUBMISSION_PATCHES_OUT = os.path.join(datasets_path, "preprocessing_results_masked","submission_patches_masked")
else:
  PATCHES_OUT = os.path.join(datasets_path, "preprocessing_results","train_patches")
  SUBMISSION_PATCHES_OUT = os.path.join(datasets_path, "preprocessing_results","submission_patches")


    
SUBMISSION_PATCH_MASKS = os.path.join(datasets_path, "preprocessing_results","submission_patches","masks")
BLURRED_PATCHES_OUT = os.path.join(datasets_path, "preprocessing_results","train_patches_blurred")
SUBMISSION_BLURRED_PATCHES_OUT = os.path.join(datasets_path, "preprocessing_results","submission_patches_blurred")
print(f"Dataset path: {datasets_path}")
print(f"Train data path: {train_data_path}")
print(f"Train labels path: {train_labels_path}")
print(f"Test data path: {test_data_path}")
print(f"Patches output path: {PATCHES_OUT}")
print(f"Submission patches output path: {SUBMISSION_PATCHES_OUT}")
print(f"Submission patch masks path: {SUBMISSION_PATCH_MASKS}")


# Define where your Ground Truth binary masks are stored
# Assuming filenames match: img_001.png -> mask_001.png (or similar)
MASKS_DIR = os.path.join(datasets_path, "preprocessing_results", "train_patches", "masks")

# ImageNet normalization statistics
IMAGENET_MEAN = [float(x) for x in [0.485, 0.456, 0.406]]  # or convert your current values to float
IMAGENET_STD  = [float(x) for x in [0.229, 0.224, 0.225]]



TARGET_SIZE = (224, 224)                    # Target size for the resized images and masks

### 3.2 Helpers

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from torchvision import transforms
from PIL import Image

def plot_sample_with_predictions(model, loader, device, label_encoder, sample_id=None, aggregation_method='max_confidence'):
    """
    Plot all patches of a single sample and the aggregated image prediction.
    (Updated for Paired Image / Two-Stream Models)
    """
    model.eval()
    
    # 1. Robust Dataset Access
    dataset = loader.dataset
    while hasattr(dataset, 'dataset'): # Unwrap Subsets
        dataset = dataset.dataset
    
    df = dataset.df
    
    # <--- CHANGED: Retrieve target_dir
    target_dir = getattr(dataset, 'target_dir', None)

    # 2. Pick a sample_id
    if sample_id is None:
        sample_id = np.random.choice(df['sample_id'].unique())
    
    # Get all patches for this sample
    sample_patches = df[df['sample_id'] == sample_id].reset_index(drop=True)
    print(f"Visualizing Sample ID: {sample_id} ({len(sample_patches)} patches)")

    # 3. Define Transforms
    # <--- CHANGED: Use the same transform (Resize+Norm) for BOTH images
    inference_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # 4. Load Data
    images_tensors = []
    targets_tensors = [] # <--- Renamed from masks_tensors
    display_imgs = []
    
    # Stats for denormalization (Display only)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)

    for _, row in sample_patches.iterrows():
        try:
            # --- A. Load Input Image ---
            img_path = row['path']
            # Handle if full path or relative
            if hasattr(dataset, 'img_dir') and dataset.img_dir:
                 img_path = os.path.join(dataset.img_dir, img_path)

            img = Image.open(img_path).convert('RGB')
            tensor_img = inference_transform(img)
            images_tensors.append(tensor_img)
            
            # Create Display Image (Denormalize -> Clamp -> Numpy)
            disp_img = torch.clamp(tensor_img * std + mean, 0, 1)
            display_imgs.append(disp_img)
            
            # --- B. Load Target Image ---
            # <--- CHANGED: Load second image based on filename match
            if target_dir:
                filename = os.path.basename(img_path)
                target_path = os.path.join(target_dir, filename)
                
                if os.path.exists(target_path):
                    target = Image.open(target_path).convert('RGB')
                else:
                    target = Image.new('RGB', (224, 224), (0, 0, 0))
            else:
                target = Image.new('RGB', (224, 224), (0, 0, 0))
            
            # <--- CHANGED: Normalize the target too
            tensor_target = inference_transform(target)
            targets_tensors.append(tensor_target)

        except Exception as e:
            print(f"Error loading {row['path']}: {e}")
            continue

    if not images_tensors:
        print("No valid images found for this sample.")
        return

    # 5. Batch Inference
    batch_imgs = torch.stack(images_tensors).to(device)
    batch_targets = torch.stack(targets_tensors).to(device)
    
    with torch.no_grad():
        # --- FIX: Pass BOTH image batches ---
        logits = model(batch_imgs, batch_targets) 
        probs = torch.softmax(logits, dim=1).cpu()

    # 6. Process Predictions
    patch_preds = probs.argmax(dim=1).numpy()
    patch_confs = probs.max(dim=1).values.numpy()

    # --- Aggregation Logic ---
    if aggregation_method == 'max_confidence':
        image_probs = probs.mean(dim=0).numpy()
        image_pred = image_probs.argmax()
        image_conf = image_probs[image_pred]
    elif aggregation_method == 'majority_voting':
        counts = np.bincount(patch_preds, minlength=len(label_encoder.classes_))
        image_pred = counts.argmax()
        image_probs = counts / counts.sum()
        image_conf = image_probs[image_pred]
    else:
        raise ValueError(f"Unknown aggregation method: {aggregation_method}")

    # Labels
    image_pred_label = label_encoder.inverse_transform([image_pred])[0]
    true_label_idx = sample_patches.iloc[0]['label_encoded']
    true_label = label_encoder.inverse_transform([true_label_idx])[0]

    # 7. Plotting
    cols = min(6, len(sample_patches))
    rows = math.ceil(len(sample_patches) / cols)
    
    fig = plt.figure(figsize=(3*cols + 4, 3*rows))
    gs = fig.add_gridspec(rows, cols + 1, width_ratios=[1]*cols + [1.3])

    # Patch grid
    for idx, (img_disp, pred, conf) in enumerate(zip(display_imgs, patch_preds, patch_confs)):
        ax = fig.add_subplot(gs[idx // cols, idx % cols])
        
        # Note: We display the INPUT image. 
        # If you want to see the target, you'd need a more complex plot (e.g., side-by-side).
        ax.imshow(img_disp.permute(1,2,0))
        
        lbl = label_encoder.inverse_transform([pred])[0]
        color = 'green' if pred == image_pred else 'red'
        
        # Title shows Patch Prediction
        ax.set_title(f"{lbl}\n{conf:.2%}", fontsize=9, color='black', backgroundcolor='white')
        ax.axis('off')

    # Aggregated Bar Chart
    ax_bar = fig.add_subplot(gs[:, -1])
    class_names = label_encoder.classes_
    colors = ['green' if i == image_pred else 'lightgray' for i in range(len(class_names))]
    
    y_pos = np.arange(len(class_names))
    ax_bar.barh(y_pos, image_probs, color=colors)
    ax_bar.set_yticks(y_pos)
    ax_bar.set_yticklabels(class_names)
    ax_bar.invert_yaxis()
    
    ax_bar.set_xlabel('Probability' if aggregation_method == 'max_confidence' else 'Vote Share')
    ax_bar.set_xlim([0,1])
    
    ax_bar.set_title(f"Sample: {sample_id}\nTrue: {true_label}\nPred: {image_pred_label} ({image_conf:.2%})\n{aggregation_method}")
    
    for i, prob in enumerate(image_probs):
        ax_bar.text(prob + 0.02, i, f"{prob:.3f}", va='center')

    plt.tight_layout()
    plt.show()

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm

def get_image_predictions(model, loader, device, method='max_mean_confidence'):
    """
    Aggregates patch-level predictions to image-level.
    Fixed for single-input models (Phikon).
    """
    model.eval()

    valid_methods = {"max_mean_confidence", "max_confidence"}
    if method not in valid_methods:
        raise ValueError(f"Unknown method '{method}'. Choose from {valid_methods}.")

    # 1. Robustly retrieve the underlying dataset
    dataset = loader.dataset
    while hasattr(dataset, 'dataset'):
        dataset = dataset.dataset
        
    df = dataset.df
    target_dir = getattr(dataset, 'target_dir', None)

    # 2. Define Transforms
    inference_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    y_true = []
    y_pred = []
    
    sample_ids = df['sample_id'].unique()

    for sample_id in tqdm(sample_ids, leave=False):
        sample_patches = df[df['sample_id'] == sample_id]

        # Ground Truth
        true_label = sample_patches.iloc[0]['label_encoded']
        y_true.append(true_label)

        patches_img = []

        for img_path in sample_patches['path']:
            try:
                # Load Input Image
                img = Image.open(img_path).convert('RGB')
                img = inference_transform(img)
                patches_img.append(img)

            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                continue

        if not patches_img:
            y_pred.append(true_label)  # Fallback if all patches fail
            continue

        # Stack batch
        batch_imgs = torch.stack(patches_img).to(device)

        with torch.no_grad():
            # ✓ Pass only the primary image
            logits = model(batch_imgs)
            
            # Softmax to get probabilities
            probs = torch.softmax(logits, dim=1)
            
            if method == "max_mean_confidence":
                # Average pooling of probabilities
                avg_probs = torch.mean(probs, dim=0)
                pred_label = torch.argmax(avg_probs).item()
            elif method == "max_confidence":
                # Max confidence pooling
                pred_label = torch.argmax(probs.max(dim=0).values).item()

        y_pred.append(pred_label)

    return y_true, y_pred



In [ ]:
def create_metadata_dataframe(patches_dir, labels_csv_path):
    """
    Creates a DataFrame mapping patch filenames to their Bag IDs and Labels.
    """
    # 1. Load the labels CSV
    # Assuming CSV structure: [image_id, label] or similar
    df_labels = pd.read_csv(labels_csv_path)

    # Standardize column names for easier merging
    # We assume the first column is the ID and the second is the Label
    id_col = df_labels.columns[0]
    label_col = df_labels.columns[1]

    # Ensure IDs in CSV are strings (to match filenames)
    df_labels[id_col] = df_labels[id_col].astype(str)

    # If the CSV IDs contain extensions (e.g., 'img_001.png'), remove them
    # because our parsed Bag IDs won't have them.
    df_labels[id_col] = df_labels[id_col].apply(lambda x: os.path.splitext(x)[0])

    # 2. List all patch files
    patch_files = [f for f in os.listdir(patches_dir) if f.endswith('.png')]

    # 3. Parse filenames to get Bag IDs
    data = []
    print(f"Found {len(patch_files)} patches. Parsing metadata...")

    for filename in patch_files:
        # Expected format from your preprocessing: {base_name}_p{i}.png
        # Example: "img_0015_p12.png" -> Bag ID should be "img_0015"

        # Split from the right on '_p' to separate Bag ID from Patch Index
        # "img_0015_p12.png" -> ["img_0015", "12.png"]
        try:
            bag_id = filename.rsplit('_p', 1)[0]

            data.append({
                'filename': filename,
                'sample_id': bag_id,
                'path': os.path.join(patches_dir, filename)
            })
        except IndexError:
            print(f"Skipping malformed filename: {filename}")

    # Create temporary patches DataFrame
    df_patches = pd.DataFrame(data)

    # 4. Merge patches with labels
    # This assigns the correct Bag Label to every Patch in that Bag
    df = pd.merge(df_patches, df_labels, left_on='sample_id', right_on=id_col, how='inner')

    # 5. Clean up and Rename
    # Keep only required columns
    df = df[['filename', label_col, 'sample_id', 'path']]

    # Rename label column to standard 'label' if it isn't already
    df = df.rename(columns={label_col: 'label'})

    print(f"Successfully created DataFrame with {len(df)} rows.")
    return df

In [ ]:
# Create a simple function to handle this
def print_model_stats(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total params: {total_params:,}")
    print(f"Trainable params: {trainable_params:,}")

## **4. Train/Val Split**

In [ ]:
patches_metadata_df = create_metadata_dataframe(PATCHES_OUT, CSV_PATH)

# Verify the result
print("\nFirst 5 rows:")
print(patches_metadata_df.head().drop(columns=['path']))
print("\nPatches per Bag (Distribution):")
print(patches_metadata_df['sample_id'].value_counts().describe())

In [ ]:
# Add Label Encoding
print("\n" + "="*50)
print("Label Encoding")
print("="*50)

label_encoder = LabelEncoder()
patches_metadata_df['label_encoded'] = label_encoder.fit_transform(patches_metadata_df['label'])

print(f"\nOriginal Labels: {label_encoder.classes_}")
print(f"Encoded as: {list(range(len(label_encoder.classes_)))}")
print(f"\nLabel Mapping:")
for orig, enc in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"  {orig} -> {enc}")

In [ ]:
# Train/Val/Test Split on Original Images (not patches)
print("\n" + "="*50)
print("Train/Val/Test Split on Original Images")
print("="*50)

# 1. Get unique sample IDs and their corresponding labels for stratification
unique_samples = patches_metadata_df['sample_id'].unique()
sample_labels = patches_metadata_df.drop_duplicates('sample_id').set_index('sample_id').loc[unique_samples, 'label_encoded']

print(f"\nTotal unique samples (original images): {len(unique_samples)}")

# 2. First Split: Train (80%) vs Temp (20%)
train_samples, temp_samples, y_train, y_temp = train_test_split(
    unique_samples,
    sample_labels,
    test_size=0.2,
    random_state=SEED,
    stratify=sample_labels
)

# 3. Second Split: Split Temp (20%) into Val (10%) and Test (10%)
# We split the temp set by 0.5 to get equal halves
val_samples, test_samples = train_test_split(
    temp_samples,
    test_size=0.5,
    random_state=SEED,
    stratify=y_temp
)

print(f"Train samples: {len(train_samples)}")
print(f"Val samples:   {len(val_samples)}")
print(f"Test samples:  {len(test_samples)}")

# 4. Create DataFrames by filtering patches based on the ID lists
df_train = patches_metadata_df[patches_metadata_df['sample_id'].isin(train_samples)].reset_index(drop=True)
df_val = patches_metadata_df[patches_metadata_df['sample_id'].isin(val_samples)].reset_index(drop=True)
df_test = patches_metadata_df[patches_metadata_df['sample_id'].isin(test_samples)].reset_index(drop=True)

# 5. Print Statistics
print(f"\nTrain patches: {len(df_train)}")
print(f"Val patches:   {len(df_val)}")
print(f"Test patches:  {len(df_test)}")

print(f"\nTrain label distribution:\n{df_train['label'].value_counts()}")
print(f"\nVal label distribution:\n{df_val['label'].value_counts()}")
print(f"\nTest label distribution:\n{df_test['label'].value_counts()}")

# Print percentage distribution
print(f"\n" + "="*50)
print("Percentage Distribution")
print("="*50)
print(f"\nTrain label percentage:\n{df_train['label'].value_counts(normalize=True) * 100}")
print(f"\nVal label percentage:\n{df_val['label'].value_counts(normalize=True) * 100}")
print(f"\nTest label percentage:\n{df_test['label'].value_counts(normalize=True) * 100}")

## **5. Transformations & Augmentation**

In [ ]:
# Define augmentation for training with enhanced transformations
train_augmentation = transforms.Compose([
    # Geometric transformations
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),  # Small rotations to handle orientation variations
    transforms.RandomAffine(
        degrees=0,
        translate=(0.1, 0.1),  # Reduced from 0.2 for more conservative shifts
        scale=None,  # Add scale variation
        shear=10  # Add shear transformation
    ),

    # Color/appearance transformations
    transforms.ColorJitter(
        brightness=0.05,  # Adjust brightness
        contrast=0.2,    # Adjust contrast
        saturation=0.2,  # Adjust saturation
        hue=0.05       # Slight hue variation
    ),
    #transforms.RandomGrayscale(p=0.1),  # Occasionally convert to grayscale to improve robustness

    # Occlusion simulation
    #transforms.RandomErasing(
    #    p=0.3,  # Reduced probability for more balanced augmentation
    #    scale=(0.02, 0.15),  # Reduced max scale
    #    ratio=(0.3, 3.3)  # Aspect ratio range
    #),

    # Optional: Add Gaussian blur for noise robustness
    # transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
])

#train_augmentation = transforms.Compose([
#    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
#    transforms.RandomHorizontalFlip(p=0.5)
#])

## **6. Custom Dataset Class**

In [ ]:
import torch
import torchvision.transforms.functional as TF
import os
import random
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class PairedTissueDataset(Dataset):
    def __init__(self, df, img_dir=None, target_dir=None, transform=None, target_size=(224, 224), label_col='label_encoded'):
        """
        Args:
            df: DataFrame containing the first image paths.
            img_dir: (Optional) Root dir to prepend to df paths.
            target_dir: Root directory where the SECOND images are located.
            transform: A torchvision.transforms object (e.g. Compose) to apply to both images.
            target_size: Tuple (height, width) to resize images to BEFORE augmentation.
        """
        self.df = df
        self.img_dir = img_dir
        self.target_dir = target_dir
        self.transform = transform
        self.target_size = target_size
        self.label_col = label_col

        # Standard Normalization (Applied individually after the synced transform)
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # --- A. Load Input Image ---
        img_path = row['path']
        if self.img_dir:
            img_path = os.path.join(self.img_dir, img_path)
            
        image = Image.open(img_path).convert("RGB")
        
        # --- B. Load Target Image ---
        filename = os.path.basename(img_path)
        
        if self.target_dir:
            target_path = os.path.join(self.target_dir, filename)
            if os.path.exists(target_path):
                target_image = Image.open(target_path).convert("RGB")
            else:
                raise FileNotFoundError(f"Target image not found: {target_path}")
        else:
            raise ValueError("target_dir must be provided.")

        # --- C. Resize (Deterministic) ---
        image = TF.resize(image, self.target_size)
        target_image = TF.resize(target_image, self.target_size)

        # --- D. SYNCHRONIZED AUGMENTATION (The "State Replay" Trick) ---
        if self.transform:
            # 1. Save the current random states
            # We save both Python's random and PyTorch's random states 
            # to ensure complete synchronization across different transform types.
            state_random = random.getstate()
            state_torch = torch.get_rng_state()

            # 2. Apply to Input Image
            image = self.transform(image)

            # 3. Restore the random states exactly as they were
            random.setstate(state_random)
            torch.set_rng_state(state_torch)

            # 4. Apply to Target Image
            # Because the RNG state is reset, the transform will make the 
            # EXACT same random choices (same crop, same flip, same autoaugment policy)
            target_image = self.transform(target_image)

        # --- E. To Tensor ---
        if not isinstance(image, torch.Tensor):
            image = TF.to_tensor(image)
        if not isinstance(target_image, torch.Tensor):
            target_image = TF.to_tensor(target_image)

        # --- F. Normalization ---
        # We normalize independently because this is a deterministic pixel operation
        image = self.normalize(image)
        target_image = self.normalize(target_image)

        # --- G. Return ---
        label = torch.tensor(row[self.label_col], dtype=torch.long)
        
        return image, label, target_image

In [ ]:
# Instantiate Datasets
train_dataset = PairedTissueDataset(df_train, transform=train_augmentation,  target_dir=BLURRED_PATCHES_OUT   )
val_dataset = PairedTissueDataset(df_val, transform=None,  target_dir=BLURRED_PATCHES_OUT   )
test_dataset = PairedTissueDataset(df_test, transform=None,  target_dir=BLURRED_PATCHES_OUT )

# Batch Size: 32 or 64 is standard for ResNet18/50 on 1MP images
BATCH_SIZE = 64

cpu_cores = os.cpu_count() or 2
num_workers = cpu_cores//2
# Instantiate Loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,          # Shuffle patches to break batch correlations
    num_workers=num_workers,         # Adjust based on your CPU
    pin_memory=True        # Faster data transfer to CUDA
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,         # No need to shuffle validation
    num_workers=num_workers,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,         # No need to shuffle test
    num_workers=num_workers,
    pin_memory=True
)

print(f"Train Loader: {len(train_loader)} batches")
print(f"Val Loader: {len(val_loader)} batches")
print(f"Num workers: {train_loader.num_workers}")

In [ ]:
def show_batch(loader, count=4):
    # 1. Robust Unpacking
    # The loader might return 2 items (Image, Label) or 3 (Image, Label, Mask)
    batch = next(iter(loader))
    images, labels = batch[0], batch[1]
    masks = batch[2] if len(batch) > 2 else None

    # Determine layout: 1 row if no masks, 2 rows if masks exist
    nrows = 2 if masks is not None else 1
    plt.figure(figsize=(15, 5 * nrows))

    # Denormalize for visualization (ImageNet stats)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

    for i in range(count):
        # --- Plot Image ---
        ax = plt.subplot(nrows, count, i + 1)
        
        img = images[i]
        img = img * std + mean  # Un-normalize
        img = torch.clamp(img, 0, 1)  # Clip to valid range
        
        plt.imshow(img.permute(1, 2, 0)) # CHW -> HWC
        plt.title(f"Label: {labels[i].item()}")
        plt.axis("off")

        # --- Plot Mask (if available) ---
        if masks is not None:
            ax = plt.subplot(nrows, count, i + 1 + count) # Second row
            mask = masks[i]
            mask = mask * std + mean  # Un-normalize
            mask = torch.clamp(mask, 0, 1)  # Clip to valid range
            
            plt.imshow(mask.permute(1, 2, 0)) # CHW -> HWC
            plt.title("Blurred Image")
            plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
print("\nVisualizing Training Batch (Augmented):")
show_batch(train_loader)

## **8. Hyperparameters**

In [ ]:
NUM_EPOCHS_TL = 5  
NUM_EPOCHS_FT = 200

PATIENCE = 6 

L2_REG = 1e-4
LR_TL = 1e-3
LR_FT = 1e-4
DROPOUT_RATE = 0.4

# --- Configuration ---
# How many blocks to unfreeze from the end of each backbone?
# 0 = Frozen, 1 = Last Block, 2 = Last 2 Blocks, etc.
UNFREEZE_LAYERS_IMG = 2  # For ResNet50 (The Color Stream)
UNFREEZE_LAYERS_MASK = 2  # For ResNet18 (The Geometry Stream)


## **9. Model Definition (Transfer Learning - PhikonV2)**

In [ ]:

# Re-define AttentionPooling (needed for PhikonClassifier)
class PhikonClassifier(nn.Module):
    def __init__(self, num_classes, model_name="owkin/phikon", dropout_rate=0.3, freeze_backbone=True):
        super().__init__()

        print(f"Loading Phikon backbone: {model_name}...")
        self.backbone = AutoModel.from_pretrained(model_name)

        # Dynamically get embedding_dim (usually 768 for ViT-Base)
        self.embedding_dim = self.backbone.config.hidden_size
        print(f"Detected backbone embedding_dim: {self.embedding_dim}")

        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        # MLP Head
        self.classifier = nn.Sequential(
            nn.Linear(self.embedding_dim, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(64, num_classes),
        )

    def forward(self, x):
        # x shape: (Batch, 3, 224, 224)
        outputs = self.backbone(pixel_values=x)
        
        # Extract the [CLS] token (index 0)
        # last_hidden_state shape: (Batch, Seq_Len, Embed_Dim) -> (Batch, Embed_Dim)
        cls_token = outputs.last_hidden_state[:, 0, :]
        
        return self.classifier(cls_token)

In [ ]:
# Assuming label_encoder is defined in your notebook scope
num_classes = len(label_encoder.classes_) # label_encoder and num_classes are from previous cells

model = PhikonClassifier(
    num_classes=num_classes,
    dropout_rate=DROPOUT_RATE
)
model = model.to(device)
print(f"Model re-initialized (Phikon ViT-G) with {num_classes} output classes and embedding_dim: {model.embedding_dim}.")


In [ ]:
print_model_stats(model)

## **10. Loss and Optimizer**

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        Args:
            alpha (Tensor, optional): Weights for each class. Shape [C].
            gamma (float): Focusing parameter. Higher value = more focus on hard examples.
                           Default is 2.0 (standard from the paper).
            reduction (str): 'mean', 'sum', or 'none'.
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # inputs: [Batch, C] (Logits)
        # targets: [Batch] (Class Indices)
        
        # 1. Standard Cross Entropy Loss (element-wise, no reduction yet)
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        
        # 2. Get the probability of the true class (pt)
        # pt = exp(-ce_loss) because ce_loss = -log(pt)
        pt = torch.exp(-ce_loss)
        
        # 3. Calculate Focal Component: (1 - pt)^gamma
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        
        # 4. Apply Class Weights (alpha) if provided
        if self.alpha is not None:
            # Gather the alpha value corresponding to the target class for each sample
            if self.alpha.device != inputs.device:
                self.alpha = self.alpha.to(inputs.device)
            
            alpha_t = self.alpha[targets]
            focal_loss = alpha_t * focal_loss

        # 5. Reduction
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [ ]:
# 1. Get Counts (from your snippet)
class_counts = df_train['label_encoded'].value_counts().sort_index().values
total_samples = sum(class_counts)
n_classes = len(class_counts)

# 2. Define Manual Tuning Factors (The "weight" knob)
# 1.0 = Default (Pure Inverse Frequency)
# > 1.0 = Force model to focus MORE on this class (e.g., critical error)
# < 1.0 = Force model to focus LESS on this class (e.g., noisy label)
# Ensure this list length matches n_classes (4 in your case)
tuning_factors = torch.tensor([1.0, 1.0, 1.0, 0.6], dtype=torch.float32)

# 3. Calculate Base Weights (Standard Inverse Frequency)
# Formula: N / (C * freq)
base_weights = torch.tensor(
    [total_samples / (n_classes * c) for c in class_counts],
    dtype=torch.float32
)

# 4. Apply Tuning
# Final Weight = Inverse_Freq_Weight * Manual_Tuning_Factor
final_weights = base_weights * tuning_factors

# 5. Move to device
final_weights = final_weights.to(device)

print(f"Base Weights:  {base_weights}")
print(f"Tuning Factors:{tuning_factors}")
print(f"Final Weights: {final_weights}")

# --- DEFINING THE LOSS ---
# Gamma=2.0 is the standard starting point.
# If your model is still ignoring the minority class, try Gamma=3.0 or 4.0.
criterion_cls = FocalLoss(alpha=None, gamma=2.3)

# Optimizer 
#optimizer = torch.optim.RAdam(
#    [p for p in model.parameters() if p.requires_grad],
#    lr=LR_TL,
#    betas=(0.9, 0.999),
#    eps=1e-8,
#    weight_decay=L2_REG
#)
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=LR_TL,
    weight_decay=L2_REG
)

## **11. Function: Training & Validation Loop**

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def calculate_metrics(preds, targets):
    # Move to CPU and convert to numpy
    preds = preds.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    # Calculate Macro F1 (balanced for all classes)
    return f1_score(targets, preds, average='macro')

In [ ]:
def train_one_epoch(model, loader, criterion_cls, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_targets = []
    
    loop = tqdm(loader, desc="Training", leave=False)
    
    for images, labels, masks in loop:
        images, labels = images.to(device), labels.to(device)
        # masks not used
        
        optimizer.zero_grad()
        
        # Pass only the image
        logits = model(images)  # ✓ Fixed
        
        loss = criterion_cls(logits, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())
        
        loop.set_postfix(loss=loss.item())
        
    epoch_loss = running_loss / len(loader)
    epoch_f1 = f1_score(all_targets, all_preds, average='macro')
    
    return epoch_loss, epoch_f1

def validate(model, loader, criterion_cls, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for images, labels, masks in loader:
            images, labels = images.to(device), labels.to(device)
            
            logits = model(images)  # ✓ Fixed
            loss = criterion_cls(logits, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
            
    epoch_loss = running_loss / len(loader)
    epoch_f1 = f1_score(all_targets, all_preds, average='macro')
    
    return epoch_loss, epoch_f1

## **12. Training Loop: Transfer Learning**

### 12.1 Training

In [ ]:

# --- Training Variables ---
best_val_f1 = 0.0
best_val_loss = float('inf')
patience_counter = 0
best_tl_epoch = 0
model_saved = False

history = {'train_loss': [], 'train_f1': [], 'val_loss': [], 'val_f1': []}

print(f"Starting Training with PhikonViTB  (Patience: {PATIENCE})...")

for epoch in range(NUM_EPOCHS_TL):
    # Train
    train_loss, train_f1 = train_one_epoch(model, train_loader, criterion_cls, optimizer, device, )

    # Validate
    val_loss, val_f1 = validate(model, val_loader, criterion_cls, device)

    # Store history
    history['train_loss'].append(train_loss)
    history['train_f1'].append(train_f1)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_f1)

    # --- Checkpointing (Save Best Model based on F1) ---
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0  # Reset counter
        best_tl_epoch = epoch + 1
        torch.save(model.state_dict(), 'models/best_model_phikonViTB_tl.pt')
        model_saved = True
    else:
        model_saved = False
        patience_counter += 1

    # Print matching your requested format
    if model_saved:
        print(f"TL Epoch {epoch+1}/{NUM_EPOCHS_TL} | Loss: {train_loss:.4f} | F1: {train_f1:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f} | Patience: {patience_counter}/{PATIENCE} Best:{best_val_f1:.4f} ✓")
    else:
        print(f"TL Epoch {epoch+1}/{NUM_EPOCHS_TL} | Loss: {train_loss:.4f} | F1: {train_f1:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f} | Patience: {patience_counter}/{PATIENCE}")

    if patience_counter >= PATIENCE:
        print("   >>> Early Stopping Triggered! Training stopped.")
        break
    
SUB_MODEL = 'models/best_model_PhikonViTB_tl.pt'
print(f"Submodel saved to {SUB_MODEL} at epoch {best_tl_epoch} with Val F1: {best_val_f1:.4f} for now. Will update if better model found in fine tuning.")
model.load_state_dict(torch.load(SUB_MODEL))

### 12.2 Training History

In [ ]:
# Plotting results
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history['train_f1'], label='Train F1 (Macro)')
plt.plot(history['val_f1'], label='Val F1 (Macro)')
plt.legend()
plt.title('F1 Score')
plt.show()

print("Best Validation F1 Score: {:.4f} at epoch {}".format(best_val_f1, best_tl_epoch))

### 12.4 Confusion Matrix

In [ ]:

# 3. Calculate and Plot Confusion Matrix
y_true_img, y_pred_img = get_image_predictions(model, val_loader, device, method="max_mean_confidence")
y_true_img_test, y_pred_img_test = get_image_predictions(model, test_loader, device, method="max_mean_confidence")

# Compute Matrix
cm_val_tl = confusion_matrix(y_true_img, y_pred_img)
cm_test_tl = confusion_matrix(y_true_img_test, y_pred_img_test)
f1_val_tl = f1_score(y_true_img, y_pred_img, average='macro')
f1_test_tl = f1_score(y_true_img_test, y_pred_img_test, average='macro')

# Plotting
plt.figure(figsize=(25, 8))
plt.subplot(1, 2, 1)
sns.heatmap(cm_val_tl, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Validation\nF1 Score: {f1_val_tl:.2f}')

plt.subplot(1, 2, 2)
sns.heatmap(cm_test_tl, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Test\nF1 Score: {f1_test_tl:.2f}')

plt.show()



## **13. Training Loop: Fine Tuning**

### 13.1 Training

In [ ]:
# 1. Initialize the NEW model instance
#    (Make sure to use the same class definition you used for training)
ft_model = PhikonClassifier(
    num_classes=num_classes,
    dropout_rate=DROPOUT_RATE,
    freeze_backbone=False
).to(device)

# 2. Load the best weights from the first phase
ft_model.load_state_dict(torch.load("models/best_model_phikonViTB_tl.pt"), strict=True)

# 3. Unfreeze parameters
#    First, ensure everything is frozen
for param in ft_model.parameters():
    param.requires_grad = False

#    Unfreeze the Classifier (Head)
for param in ft_model.classifier.parameters():
    param.requires_grad = True


#    Unfreeze the last 2 blocks of the Backbone (Features)
#    For a Vision Transformer (ViT) like Phikon, the layers are in `backbone.encoder.layer`
for block in ft_model.backbone.encoder.layer[-2:]:
    for param in block.parameters():
        param.requires_grad = True

# Optimizer
ft_optimizer = torch.optim.RAdam(
    [p for p in ft_model.parameters() if p.requires_grad],
    lr=1e-3,
    weight_decay=L2_REG
)

# 5. New Scheduler for the new optimizer
ft_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    ft_optimizer, mode='min', factor=0.1, patience=3,
)


In [ ]:
print("Starting Fine-Tuning Phase...")

# Reset history
ft_history = {'train_loss': [], 'train_f1': [], 'val_loss': [], 'val_f1': []}
best_val_f1_ft = 0.0
patience_counter_ft = 0
best_ft_epoch = 0
model_saved = False
ft_better_than_tl = False

# --- 6. Fine-Tuning Loop ---
for epoch in range(NUM_EPOCHS_FT):
    # Train (Pass BOTH criteria)
    train_loss, train_f1 = train_one_epoch(ft_model, train_loader, criterion_cls , ft_optimizer, device)

    # Validate (Pass BOTH criteria)
    val_loss, val_f1 = validate(ft_model, val_loader, criterion_cls, device)

    # Update Scheduler based on Validation Loss
    ft_scheduler.step(val_loss)

    # Store history
    ft_history['train_loss'].append(train_loss)
    ft_history['train_f1'].append(train_f1)
    ft_history['val_loss'].append(val_loss)
    ft_history['val_f1'].append(val_f1)

    # --- Checkpointing & Early Stopping ---
    if val_f1 > best_val_f1_ft:
        best_val_f1_ft = val_f1
        best_ft_epoch = epoch + 1
        patience_counter_ft = 0 
        
        # Save the fine-tuned model
        torch.save(ft_model.state_dict(), 'models/best_model_PhikonViTB_ft.pt')
        model_saved = True
        
        # Check if we beat the Transfer Learning phase
        if best_val_f1_ft > best_val_f1:
            ft_better_than_tl = True
    else:
        model_saved = False
        patience_counter_ft += 1

    # Print Status
    status_mark = "✓" if model_saved else ""
    print(f"FT Epoch {epoch+1}/{NUM_EPOCHS_FT} | Loss: {train_loss:.4f} | F1: {train_f1:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f} | Patience: {patience_counter_ft}/{PATIENCE} {status_mark}")

    if patience_counter_ft >= PATIENCE:
        print(f"\nEarly Stopping Triggered! Best FT Epoch: {best_ft_epoch} with Val F1: {best_val_f1_ft:.4f}")
        break

# Final Summary
SUB_MODEL = 'models/best_model_PhikonViTB_ft.pt' if ft_better_than_tl else 'models/best_model_PhikonViTB_tl.pt'
print(f"\nFine-Tuning Complete.")
print(f"Best TL F1: {best_val_f1:.4f}")
print(f"Best FT F1: {best_val_f1_ft:.4f}")
print(f"Best Model Saved to: {SUB_MODEL}")

### 13.2 Training History

In [ ]:
# Plotting results
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
plt.plot(ft_history['train_loss'], label='Train Loss')
plt.plot(ft_history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(ft_history['train_f1'], label='Train F1 (Macro)')
plt.plot(ft_history['val_f1'], label='Val F1 (Macro)')
plt.legend()
plt.title('F1 Score')
plt.show()

print("Best Fine-Tuned Validation F1 Score: {:.4f} at epoch {}".format(best_val_f1, best_ft_epoch))

### 13.3 Confusion Matrix

In [ ]:
y_true_ft, y_pred_ft = get_image_predictions(ft_model, val_loader, device, method="max_confidence")
y_true_ft_test, y_pred_ft_test = get_image_predictions(ft_model, test_loader, device, method="max_confidence")

cm_val_ft = confusion_matrix(y_true_ft, y_pred_ft)
cm_test_ft = confusion_matrix(y_true_ft_test, y_pred_ft_test)

f1_val_ft = f1_score(y_true_ft, y_pred_ft, average='macro')
f1_test_ft = f1_score(y_true_ft_test, y_pred_ft_test, average='macro')


In [ ]:
# Create a 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
axes = axes.flatten()  # Flatten to 1D array for easy iteration

# Prepare data lists for iteration
# Order: T1 Val, T1 Test, FT Val, FT Test
cms = [cm_val_tl, cm_test_tl, cm_val_ft, cm_test_ft]
f1_scores = [f1_val_tl, f1_test_tl, f1_val_ft, f1_test_ft]
titles = [
    "Transfer Learning (Validation)", 
    "Transfer Learning (Test)", 
    "Fine-Tuning (Validation)", 
    "Fine-Tuning (Test)"
]

for i, ax in enumerate(axes):
    sns.heatmap(
        cms[i], 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=label_encoder.classes_,
        yticklabels=label_encoder.classes_,
        ax=ax
    )
    
    # Set title with F1 score included
    ax.set_title(f"{titles[i]}\nF1 Score: {f1_scores[i]:.4f}", fontsize=14, fontweight='bold')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')

plt.tight_layout()
plt.show()

## **14. Submission Creation**

### 14.1 Create Submission Model

In [ ]:
sub_model =PhikonClassifier(
    num_classes=num_classes,
    model_name="owkin/phikon-v2",  # Using ViT-G model
    dropout_rate=DROPOUT_RATE
).to(device)
# 2. Load the best weights from the first phase
sub_model.load_state_dict(torch.load('models/best_model_PhikonViTB_ft.pt'), strict=True)

In [ ]:
def generate_submission(
    model,
    submission_folder,
    label_encoder,
    target_folder=None,
    method='max_confidence',
    output_csv="submission.csv",
    device='cuda',
    two_stream=False,
):
    """
    Generates a submission file for single-stream or two-stream models.

    Args:
        model: Trained model. Can be:
               - Single-stream (e.g., PhikonClassifier): forward(self, x)
               - Two-stream (e.g., TwoStream/ResNetParallel): forward(self, x1, x2)
        submission_folder: Folder containing test patch images (Input 1).
        label_encoder: The LabelEncoder object used during training to decode predictions.
        target_folder: (Optional) Folder containing corresponding second images (Input 2).
                       If None and two_stream=True, uses black images as second stream.
        method: 'max_confidence' or 'majority_voting' for aggregating patch predictions.
        output_csv: Path to save the submission CSV.
        device: 'cuda' or 'cpu'.
        two_stream: If True, expects model(x1, x2). If False, expects model(x1).
    """
    model.eval()
    model.to(device)  # Ensure model is on the correct device

    # 1. Get list of test patches
    patch_files = sorted([
        f for f in os.listdir(submission_folder)
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ])

    print(f"Found {len(patch_files)} patches in {submission_folder}")

    if two_stream:
        if target_folder:
            print(f"Two-stream mode: looking for second images in: {target_folder}")
        else:
            print("Two-stream mode: no target folder provided. Using black images as second stream.")
    else:
        if target_folder:
            print("Single-stream mode: target_folder provided but will be ignored.")
        else:
            print("Single-stream mode: using only primary images.")

    # 2. Define Transforms (Must match training/validation transforms)
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Store predictions per image
    image_predictions = {}
    missing_targets_count = 0

    print("Running inference...")
    with torch.no_grad():
        for filename in tqdm(patch_files):
            filepath = os.path.join(submission_folder, filename)

            try:
                # --- ID Extraction ---
                # Extracts "img_123" from "img_123_patch_0.png" or "img_123_p1.png"
                if '_p' in filename:
                    sample_id = filename.rsplit('_p', 1)[0]
                elif '_patch' in filename:
                    sample_id = filename.rsplit('_patch', 1)[0]
                else:
                    # Fallback for whole images
                    sample_id = os.path.splitext(filename)[0]

                if sample_id not in image_predictions:
                    image_predictions[sample_id] = {'probs': []}

                # --- A. Load Input Image ---
                image = Image.open(filepath).convert('RGB')
                input_tensor = val_transform(image).unsqueeze(0).to(device)

                # --- B. Load Second Image (Target) if two_stream ---
                if two_stream:
                    # Default: Black RGB image
                    target_pil = Image.new('RGB', image.size, (0, 0, 0))
                    
                    if target_folder:
                        # Look for the EXACT same filename in the target folder
                        target_path = os.path.join(target_folder, filename)
                        
                        if os.path.exists(target_path):
                            target_pil = Image.open(target_path).convert('RGB')
                        else:
                            missing_targets_count += 1
                    
                    # Apply normalization to target as well
                    target_tensor = val_transform(target_pil).unsqueeze(0).to(device)

                    # --- C. Model Inference (Two-Stream) ---
                    logits = model(input_tensor, target_tensor)
                else:
                    # --- C. Model Inference (Single-Stream) ---
                    logits = model(input_tensor)

                # Softmax to get probabilities
                probs = torch.softmax(logits, dim=1)

                image_predictions[sample_id]['probs'].append(
                    probs.cpu().numpy()[0]
                )

            except Exception as e:
                print(f"Error processing {filename}: {e}")
                pass
    
    if two_stream and target_folder and missing_targets_count > 0:
        print(f"WARNING: Could not find paired images for {missing_targets_count} patches.")

    # 3. Aggregate Results
    final_results = []

    print(f"Aggregating results for {len(image_predictions)} unique samples...")

    for sample_id, data in image_predictions.items():
        all_probs = np.array(data['probs']) 

        # Construct sample name (adjust extension if needed, e.g. .tif or .png)
        # If your submission file requires IDs like "10045", remove the extension part below.
        sample_index_name = f"{sample_id}.png"

        if len(all_probs) == 0:
            # Fallback (should not happen if loop ran)
            final_results.append({
                'sample_index': sample_index_name,
                'label': "Luminal A"
            })
            continue

        if method == 'majority_voting':
            patch_preds = np.argmax(all_probs, axis=1)
            counts = np.bincount(patch_preds)
            final_class_idx = np.argmax(counts)

        elif method == 'max_confidence':
            # Average the probabilities across all patches for this slide
            avg_probs = np.mean(all_probs, axis=0)
            final_class_idx = np.argmax(avg_probs)
        else:
            raise ValueError(f"Unknown aggregation method: {method}")

        # Decode Label using the PASSED label_encoder
        pred_label = label_encoder.inverse_transform([final_class_idx])[0]
        final_results.append({
            'sample_index': sample_index_name,
            'label': pred_label
        })

    # 4. Save
    df_submission = pd.DataFrame(final_results)
    
    if 'sample_index' in df_submission.columns:
        df_submission.sort_values('sample_index', inplace=True)
    
    df_submission.to_csv(output_csv, index=False)

    print(f"Submission saved to {output_csv}")

    return df_submission


In [ ]:
def plot_submission_distribution(submission_data, output_file="prediction_distribution.png", ax=None):
    """
    Plots the distribution of predicted labels.

    Args:
        submission_data: DataFrame or path to csv.
        output_file: Path to save the plot (only used if ax is None).
        ax: Matplotlib axes object. If None, a new figure is created.
    
    Returns:
        ax: The matplotlib Axes object.
    """
    # 1. Load Data
    if isinstance(submission_data, str):
        if not os.path.exists(submission_data):
            print(f"Error: File {submission_data} not found.")
            return None
        df = pd.read_csv(submission_data)
    elif isinstance(submission_data, pd.DataFrame):
        df = submission_data
    else:
        raise ValueError("Input must be a pandas DataFrame or a file path string.")

    if 'label' not in df.columns:
        print("Error: DataFrame must contain a 'label' column.")
        return None

    # 2. Setup Plot Style
    sns.set_theme(style="whitegrid")
    
    # Check if external axes provided
    is_standalone = False
    if ax is None:
        is_standalone = True
        fig, ax = plt.subplots(figsize=(10, 6))

    # 3. Create Count Plot
    order = df['label'].value_counts().index
    sns.countplot(x='label', data=df, order=order, palette="viridis", hue='label', legend=False, ax=ax)

    # 4. Add Annotations
    total = len(df)
    for p in ax.patches:
        height = p.get_height()
        if height > 0: # Check to avoid errors with empty bars
            percentage = '{:.1f}%'.format(100 * height / total)
            ax.annotate(f'{int(height)}\n({percentage})',
                        (p.get_x() + p.get_width() / 2., height),
                        ha='center', va='bottom', 
                        fontsize=11, color='black', xytext=(0, 5),
                        textcoords='offset points')

    # 5. Formatting
    ax.set_title(f'Prediction Distribution (N={total})', fontsize=15, fontweight='bold')
    ax.set_xlabel('Predicted Class', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.tick_params(axis='x', rotation=45)

    # 6. Save and Show (Only if standalone)
    if is_standalone:
        plt.tight_layout()
        plt.show()

    return ax

### 14.3 Create the Submission CSV

In [ ]:
# Example Usage:
# Replace 'path/to/SUBMISSION_PATCHES' with the actual path
# If your folder is just named SUBMISSION_PATCHES in current dir:

now = datetime.now()
date_time_str = now.strftime("%d_%b-%H_%M")

sub_dir = os.path.join(os.path.pardir, "submission_csvs")
OUTPUT_NAME = os.path.join(sub_dir, f"submission_ft--{date_time_str}.csv")


os.makedirs(sub_dir, exist_ok=True)

# Check if folder exists
if os.path.exists(SUBMISSION_PATCHES_OUT):
    # Method 1: Max Confidence / Average Probability (Recommended)
    df_sub_max_conf = generate_submission(sub_model, SUBMISSION_PATCHES_OUT, method='max_confidence', output_csv=OUTPUT_NAME, label_encoder=label_encoder)
    print(f"Submission CSV saved to: {OUTPUT_NAME}")

    # Method 2: Majority Voting (Optional, uncomment to run)
    # df_sub_majority_voting = generate_submission(sub_model, SUBMISSION_PATCHES_OUT, method='majority_voting', output_csv="submission_voting.csv")
else:
    print(f"Directory '{SUBMISSION_PATCHES_OUT}' not found. Please create it or set the correct path.")

In [ ]:

# Create a figure with 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot on specific axes
plot_submission_distribution(df_sub_max_conf, ax=axes[0])
axes[0].set_title("Submission") # Override title if desired

plot_submission_distribution(CSV_PATH, ax=axes[1])
axes[1].set_title("Train Data")

plt.tight_layout()
plt.show()